In [21]:
import glob
import math
from matplotlib import pyplot as plt
import napari
import numpy as np
from numpy import asarray

from skimage.io import imread,imshow,imsave
from skimage.filters import threshold_otsu
from skimage.measure import regionprops_table
import os
import pandas as pd
import PIL
from PIL import Image
import tifffile

from skimage.color import label2rgb
from skimage.segmentation import expand_labels,find_boundaries, mark_boundaries
os.chdir ('/Users/kim/Data/CRUK-OHSU_onedrive final 2022-09-30/Manuscript/IMC-IF method paper/Cell_report_method_resource/github')

print('library import done')

library import done


In [22]:
df=pd.read_csv('Analysis/new_July_2021/IFtilemask/IFtilelist.csv') # tile mask metadata 
df=df.sort_values('tileorder')
df['tileorder'][0]

'Analysis/new_July_2021/IFtilemask/DE18.NE_x_0_y_00.tif_mask.png'

In [23]:
slides=df['name'].drop_duplicates().tolist() # Whole slide image name 
slides

['DE18.NE', 'DE28.NE', 'PS14.24197-1E', 'PS15.19650-B3', 'PS16.38634-F2']

In [27]:
inputfolder='IFtilemask_mesmer'
preoutputfolder='maskmerge_mesmer_org'
outputfolder='maskmerge_mesmer_0325'

# Next 
# inputfolder='IFtilemask_mesmer_nuc'  
# preoutputfolder='maskmerge_mesmer_nuc_org' 
# outputfolder='maskmerge_mesmer_nuc_0325'  

mask_tile_list=sorted(glob.glob(f'Analysis/new_July_2021/{inputfolder}/*')) # mask tile location. 

In [28]:
for a in mask_tile_list:
    im=imread(a)
    print(a)
    print(im.shape)
    print(im.dtype)
print('done')

done


In [29]:
n=0
for slide in slides: #slide name 
    name=slide
    masks = []
    for lst in mask_tile_list: # slide name - mask tile sorting. 
        if name in lst:
            masks.append(lst)
    
    # ROI tile composition check 
    xnum=int(masks[-1].split('/')[-1].split('x_')[-1].split('_y_')[0])+1
    ynum=int(masks[-1].split('/')[-1].split('_y_')[-1].split('.tif')[0])+1

    # Mask array list   
    maskarrays=[np.array(imread(mask),np.uint32) for mask in masks]
    
    # Mask intensity renewal 
    prevmax = 0 
    inc_maskarrays=[]  
    for m in maskarrays:
        update= np.where(m>0,m+prevmax,0)
        inc_maskarrays.append(update)
        if m.any()> 0:
            prevmax=np.max(update)
        else:
            continue

    ### Mask intensity increase    
    start=0

    for array in inc_maskarrays:
        if np.max(array)>start:
            start=np.max(array)
    print("total cell number of ",name, "is", np.max(start))
    
    # Mask merge
    merge=np.vstack([np.hstack([inc_maskarrays[i*ynum+j] for j in range(ynum)]) for i in range(xnum)])
    tifffile.imsave(f'Analysis/new_July_2021/{preoutputfolder}/{name}.tiff',merge) # input merged mask folder location. 
    img = Image.fromarray(merge)

    # Mask resize 
    width=math.ceil(merge.shape[0]*0.325)
    height=math.ceil(merge.shape[1]*0.325)    
    imgresize=img.resize(size=(height,width),resample=PIL.Image.NEAREST)
    pix = np.array(imgresize)
    
    tifffile.imsave(f'Analysis/new_July_2021/{outputfolder}/{name}.tiff',pix) # input merged mask folder location. 
    image = imread(f'Analysis/new_July_2021/{outputfolder}/{name}.tiff') # for double-checking 
        
    print(name, f":{len(inc_maskarrays)} files loading done",'xnum:',xnum, 'ynum:',ynum)
    print('merged mask size:', merge.shape)
    print('image size after resizing:', pix.shape)
    print('cell number after resizing:', np.max(pix))
    print('image size after resizing:', pix.shape)
    print('image type after resizing:',image.dtype)
    print('')

    n=n+1
print('done')

total cell number of  DE18.NE is 129865
DE18.NE :96 files loading done xnum: 8 ynum: 12
merged mask size: (35171, 57206)
image size after resizing: (11431, 18592)
cell number after resizing: 129865
image size after resizing: (11431, 18592)
image type after resizing: int32

total cell number of  DE28.NE is 60735
DE28.NE :40 files loading done xnum: 8 ynum: 5
merged mask size: (38814, 22319)
image size after resizing: (12615, 7254)
cell number after resizing: 60735
image size after resizing: (12615, 7254)
image type after resizing: int32

total cell number of  PS14.24197-1E is 85099
PS14.24197-1E :24 files loading done xnum: 8 ynum: 3
merged mask size: (38828, 13103)
image size after resizing: (12620, 4259)
cell number after resizing: 85099
image size after resizing: (12620, 4259)
image type after resizing: int32

total cell number of  PS15.19650-B3 is 125875
PS15.19650-B3 :45 files loading done xnum: 9 ynum: 5
merged mask size: (42517, 20448)
image size after resizing: (13819, 6646)
cel

### Merged File size Check

In [32]:
# PS14
IForgpath='Analysis/new_July_2021/CZItoTiff/IF/PS14.24197-1E.czi.tif'
maskmaergepath='Analysis/new_July_2021/maskmerge_mesmer_0325/PS14.24197-1E.tiff'
IMCregpath='Analysis/new_July_2021/IMC_Registration/registered_ROI001_023_PS14.24197-1E/Registered_191Ir_Ir191_DNA.ome.tiff'

viewer = napari.Viewer()

# IF original
image=imread(IForgpath)
viewer.add_image(image, name='IForiginal',blending='additive');
print("Original IF image sizse is:", image.shape)

# IFresize
image=imread(IForgpath)
width=math.ceil(image.shape[0]*0.325)
height=math.ceil(image.shape[1]*0.325)
img = Image.fromarray(image[:,:,0])
imgresize=img.resize(size=(height,width),resample=PIL.Image.NEAREST)
pix = np.array(imgresize)
viewer.add_image(pix, name='IFimgresize',blending='additive',colormap='green');
print("Resized IF image sizse is:", pix.shape)

# Maskmerge
maskimage=imread(maskmaergepath)
viewer.add_image(maskimage, name='Maskmerge',blending='additive',colormap='yellow');
print("Merged maskimage sizse is:", maskimage.shape)

nuc_marked = mark_boundaries(maskimage, maskimage, color=(1,0,1))
viewer.add_image(nuc_marked, name='Maskbound',blending='additive');

# IMCreg 
image=imread(IMCregpath)
viewer.add_image(image, name='IMCreg',blending='additive',colormap='red');
print("Registered IMC image sizse is:", image.shape)

Original IF image sizse is: (38828, 13103, 3)
Resized IF image sizse is: (12620, 4259)
Merged maskimage sizse is: (12620, 4259)
Registered IMC image sizse is: (12620, 4259)
